In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.neural_network import MLPRegressor
pd.set_option('display.max_columns', None)

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Training set
ts_i = pd.read_csv('./Data/features_inter.csv')
ts_u = pd.read_csv('./Data/features_union.csv')
ts_i = ts_i.drop(['parcelid'],axis=1)
ts_u = ts_u.drop(['parcelid'],axis=1)
ts_i.head()

,logerror,finishedsquarefeet12,lotsizesquarefeet,taxamount,longitude,latitude,regionidzip,yearbuilt,month
0,0.0276,1684.0,7528.0,6735.880000,-118.488536,34.280990,96370.0,1959.0,1
1,-0.1684,2263.0,3643.0,10153.020000,-117.677556,33.668120,96962.0,2014.0,1
2,-0.0040,2217.0,11423.0,5984.582647,-118.175032,34.136312,96293.0,1940.0,1
3,0.0218,839.0,70859.0,3048.740000,-118.309000,33.755800,96222.0,1987.0,1
4,-0.0050,2283.0,6000.0,5488.960000,-117.700234,33.485643,96961.0,1981.0,1


In [3]:
def mae_loss(t,p):
    sum = 0
    for i in range(t.shape[0]):
        sum += abs(t[i]-p[i])
    return sum/t.shape[0]
def metric_nn(X_train, y_train, X_test, y_test, model, cri):
    print(model)
    fit_time = 0
    score_time = 0
    test_score = 0
    train_score = 0
    result = cross_validate(model, X_test, y_test, scoring=cri)
    for i in range(3):
        fit_time += result['fit_time'][i]
        score_time += result['score_time'][i]
        test_score += result['test_score'][i]
        train_score += result['train_score'][i]
    pred = model.predict(X_test)
    m = mae_loss(y_test, pred)
    return [fit_time/3, score_time/3, test_score/3, train_score/3, m]

In [4]:
# Random Forest with MSE
ts_list = [ts_i,ts_u]
model_list = []
frames = []
for m in ts_list:
    for i in range(5):
        for j in range(5):
            print("Round %d:" % (i*5+j+1))
            
            # Training set
            ts = m
            y = ts[['logerror']]
            X = ts.drop('logerror',axis=1)
            
            # Split data with 0.75
            X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)
            print (X_train.shape)
            print (y_train.shape)
            print (X_test.shape)
            print (y_test.shape)
            
            nn = MLPRegressor(hidden_layer_sizes=(i*5+j+1,), alpha=0.0001*(i*5+j+1))
            nn.fit(X_train, y_train)
            model_list.append(nn)
            metric = metric_nn(X_train, np.array(y_train), X_test, np.array(y_test), nn, 'neg_mean_squared_error')
            print(metric)
            result = pd.DataFrame.from_items([(i*5+j,metric)],\
                                             orient='index',\
                                             columns=['fit_time','score_time','test_score','train_score','MAE'])
            frames.append(result)
            print("****************************")
m = pd.concat(frames)
print(m)

Round 1:
(115563, 8)
(115563, 1)
(49528, 8)
(49528, 1)


/usr/local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)


/usr/local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0.86398967107137048, 0.0035357475280761719, -25.579581705518596, -26.03896435344792, array([ 0.04405739])]
****************************
Round 2:
(115563, 8)
(115563, 1)
(49528, 8)
(49528, 1)
MLPRegressor(activation='relu', alpha=0.0002, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(2,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)


...

[0.58773962656656897, 0.0087486902872721348, -10100789746.384579, -9949457028.8000622, array([ 3617.76596895])]
****************************
Round 24:
(115563, 11)
(115563, 1)
(49528, 11)
(49528, 1)
MLPRegressor(activation='relu', alpha=0.0024000000000000002,
       batch_size='auto', beta_1=0.9, beta_2=0.999, early_stopping=False,
       epsilon=1e-08, hidden_layer_sizes=(24,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)


/usr/local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0.68154668807983398, 0.0085096359252929688, -9259276006712.2012, -9276170942429.502, array([ 10787.12838878])]
****************************
Round 25:
(115563, 11)
(115563, 1)
(49528, 11)
(49528, 1)
MLPRegressor(activation='relu', alpha=0.0025, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(25,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)


/usr/local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0.74598399798075354, 0.010065952936808268, -420514677.32030231, -398780205.35880762, array([ 11626.08450767])]
****************************
    fit_time  score_time    test_score   train_score                MAE
0   0.863990    0.003536 -2.557958e+01 -2.603896e+01  [0.0440573894209]
1   4.698045    0.005328 -1.713937e-01 -5.085197e-03  [0.0914471033692]
2   2.896818    0.003750 -1.026078e+03 -9.188533e+02   [0.561921028892]
3   4.287822    0.006343 -7.766427e+02 -2.973693e+02    [1.70453952273]
4   1.545740    0.003524 -2.139918e+03 -8.046787e+02    [8.74539688771]
5   2.525170    0.013028 -1.971529e+03 -1.551721e+03    [6.65979789194]
6   1.472715    0.005747 -7.488922e+04 -7.781066e+04    [4.22856882653]
7   2.410066    0.005515 -2.341965e+04 -1.068985e+04    [95.8946319174]
8   3.080602    0.005206 -1.037318e+04 -1.112778e+04    [43.4895796101]
9   1.382309    0.004678 -1.148633e+03 -1.404226e+03    [7.62735167525]
10  1.598298    0.005138 -1.833488e+04 -1.611113e+04    [79.7222081

In [6]:
m.sort_values(by='MAE')

,fit_time,score_time,test_score,train_score,MAE
0,0.863990,0.003536,-2.557958e+01,-2.603896e+01,[0.0440573894209]
6,0.308198,0.004650,-9.790713e+08,-1.055294e+09,[0.0477147771863]
1,4.698045,0.005328,-1.713937e-01,-5.085197e-03,[0.0914471033692]
4,1.823143,0.005034,-7.727404e+08,-9.472242e+08,[0.178265863157]
2,2.896818,0.003750,-1.026078e+03,-9.188533e+02,[0.561921028892]
0,2.005757,0.001585,-7.876879e-03,-7.786976e-03,[0.839432117493]
3,4.287822,0.006343,-7.766427e+02,-2.973693e+02,[1.70453952273]
6,1.472715,0.005747,-7.488922e+04,-7.781066e+04,[4.22856882653]
12,1.303903,0.005353,-4.877033e+04,-5.848572e+04,[5.31481420505]
5,2.525170,0.013028,-1.971529e+03,-1.551721e+03,[6.65979789194]


In [21]:
# best = model_list[2]
# X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)
# pred = best.predict(X_test)
# error = pred - y_test